In [1]:
from pathlib import Path
import torch
import matplotlib.pyplot as plt
import torch

from torch import nn
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torchvision import transforms

from torchvision.models import EfficientNet_V2_S_Weights, efficientnet_v2_s
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import inception_v3, Inception_V3_Weights

from torchvision import datasets

from torch.utils.data import DataLoader

from torch import nn
from timeit import default_timer as timer

from going_modular.going_modular import engine

In [2]:
lr1 = 0.01
lr2 = 0.001
step_size = 10
patience = 3


In [3]:
data_path = Path("data/")
image_path = data_path / "events"
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir
output_dir = 'output'

In [4]:

torch.cuda.empty_cache()
torch.manual_seed(42)
torch.cuda.manual_seed(42)
device = torch.device("cuda:0")

In [5]:
weights = EfficientNet_V2_S_Weights.DEFAULT
model = efficientnet_v2_s(weights=weights).to(device)

# weights = ResNet50_Weights.DEFAULT
# model =resnet50(weights=weights).to(device)

# weights = Inception_V3_Weights.DEFAULT
# model = inception_v3(weights=weights).to(device)

In [6]:
weights.transforms()

ImageClassification(
    crop_size=[384]
    resize_size=[384]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [7]:
auto_transforms = weights.transforms()
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    # transforms.Resize(112),  # Downscale
    # transforms.Resize(224),
    # transforms.RandomRotation(20),
    # transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    # transforms.CenterCrop(224), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [8]:
# train_data = datasets.ImageFolder(root=train_dir, # target folder of images
#                                   transform=auto_transforms, # transforms to perform on data (images)
#                                   target_transform=None) # transforms to perform on labels (if necessary)

train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=transform, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)

test_data = datasets.ImageFolder(root=test_dir,
                                 transform=auto_transforms)


In [9]:

class_names = train_data.classes
class_names

['Combat',
 'DestroyedBuildings',
 'Fire',
 'Humanitarian Aid and rehabilitation',
 'Military vehicles and weapons']

In [10]:
train_dataloader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=32,shuffle=False)

In [11]:
# Get the length of class_names (one output unit for each class)
output_shape = len(class_names)
# Recreate the classifier layer and seed it to the target device

model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.1, inplace=True),
    nn.Linear(in_features=1280, out_features=output_shape, bias=True),
).to(device)

In [12]:
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1) # this is also called "criterion"/"cost function" in some places
optimizer = torch.optim.SGD(model.parameters(), lr=lr1, momentum=0.9, weight_decay=0.01)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=step_size, gamma=0.1)
# warmup_scheduler = WarmupLR(scheduler, warmup_factor=0.1, warmup_iters=10, warmup_method="linear")


In [13]:
torch.cuda.set_device(0)

In [14]:
next(model.parameters()).is_cuda

True

In [15]:
for layer in model.features.parameters():
    layer.requires_grad = False

# for layer in model.parameters():
#     layer.requires_grad = True


In [16]:
# Set the random seeds


# Start the timer
start_time = timer()

# Setup training and save the results
results = engine.train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       scheduler=scheduler,
                       loss_fn=loss_fn,
                       epochs=50,
                       device=device,
                       patience=patience)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")


  0%|          | 0/50 [00:00<?, ?it/s]

b:\Softwares\anaconda3\envs\GG_2527\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch: 1 | train_loss: 1.4612 | train_acc: 0.4375 | test_loss: 1.0924 | test_acc: 0.9271
Epoch: 2 | train_loss: 0.9149 | train_acc: 0.8500 | test_loss: 0.6149 | test_acc: 0.9583
Epoch: 3 | train_loss: 0.7509 | train_acc: 0.8594 | test_loss: 0.5761 | test_acc: 0.9271
Epoch: 4 | train_loss: 0.6661 | train_acc: 0.8906 | test_loss: 0.5646 | test_acc: 0.9479
Epoch: 5 | train_loss: 0.6429 | train_acc: 0.9031 | test_loss: 0.5742 | test_acc: 0.8958
Epoch: 6 | train_loss: 0.6362 | train_acc: 0.9187 | test_loss: 0.5719 | test_acc: 0.8958
Epoch: 7 | train_loss: 0.6354 | train_acc: 0.9094 | test_loss: 0.5635 | test_acc: 0.9167
Epoch: 8 | train_loss: 0.6284 | train_acc: 0.9031 | test_loss: 0.5639 | test_acc: 0.9167
Epoch: 9 | train_loss: 0.6166 | train_acc: 0.9156 | test_loss: 0.5422 | test_acc: 0.9688
Epoch: 10 | train_loss: 0.6188 | train_acc: 0.9094 | test_loss: 0.5483 | test_acc: 0.9479
Epoch: 11 | train_loss: 0.6052 | train_acc: 0.9219 | test_loss: 0.5434 | test_acc: 0.9688
Epoch: 12 | train_l

In [17]:
# for layer in model.features.parameters():
#     layer.requires_grad = True


for layer in model.parameters():
    layer.requires_grad = True

In [18]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr2, momentum=0.9)
scheduler = StepLR(optimizer, step_size=step_size, gamma=0.1)
patience = 3

In [19]:
start_time = timer()

# Setup training and save the results
results = engine.train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       scheduler=scheduler,
                       loss_fn=loss_fn,
                       epochs=15,
                       device=device,
                       patience=patience)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")


  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.5936 | train_acc: 0.9187 | test_loss: 0.5324 | test_acc: 0.9583
Epoch: 2 | train_loss: 0.5924 | train_acc: 0.9250 | test_loss: 0.5319 | test_acc: 0.9583
Epoch: 3 | train_loss: 0.5951 | train_acc: 0.9156 | test_loss: 0.5320 | test_acc: 0.9688
Epoch: 4 | train_loss: 0.5756 | train_acc: 0.9437 | test_loss: 0.5262 | test_acc: 0.9688
Epoch: 5 | train_loss: 0.5466 | train_acc: 0.9531 | test_loss: 0.5270 | test_acc: 0.9479
Epoch: 6 | train_loss: 0.5720 | train_acc: 0.9375 | test_loss: 0.5261 | test_acc: 0.9479
Epoch: 7 | train_loss: 0.5932 | train_acc: 0.9156 | test_loss: 0.5299 | test_acc: 0.9583
Epoch: 8 | train_loss: 0.5312 | train_acc: 0.9719 | test_loss: 0.5262 | test_acc: 0.9688
Epoch: 9 | train_loss: 0.5256 | train_acc: 0.9719 | test_loss: 0.5207 | test_acc: 0.9792
Epoch: 10 | train_loss: 0.5482 | train_acc: 0.9500 | test_loss: 0.5142 | test_acc: 0.9688
Epoch: 11 | train_loss: 0.5443 | train_acc: 0.9469 | test_loss: 0.5228 | test_acc: 0.9688
Epoch: 12 | train_l

In [20]:
torch.save(model.state_dict(), 'w.tf')

In [21]:
# model_res = resnet50().to(device)
# model_res.classifier = torch.nn.Sequential(
#     nn.Dropout(p=0.2, inplace=True),
#     nn.Linear(in_features=1280, out_features=5, bias=True),
# ).to(device)
# model_res.load_state_dict(torch.load('resnet.tf'))
# model_res.eval()

In [22]:
# weights_eff = EfficientNet_V2_S_Weights.DEFAULT
# model_eff = efficientnet_v2_s().to(device)
# model_eff.classifier = torch.nn.Sequential(
#     nn.Dropout(p=0.2, inplace=True),
#     nn.Linear(in_features=1280, out_features=5, bias=True),
# ).to(device)
# model_eff.load_state_dict(torch.load('weights_copy.tf'))
# model_eff.eval()

In [23]:
# class EnsembleModel(nn.Module):   
#     def __init__(self, modelA, modelB):
#         super().__init__()
#         self.modelA = modelA
#         self.modelB = modelB
#         self.classifier = nn.Linear(1005, 200)
        
#     def forward(self, x):
#         x1 = self.modelA(x)
#         x2 = self.modelB(x)
#         x = torch.cat((x1, x2), dim=1)
#         out = self.classifier(x)
#         return out
    
# ensemble_model = EnsembleModel(model_eff, model_res)

In [24]:
# import numpy as np

# start_time = timer()

# # Setup training and save the results
# ensemble_training_results =  engine.train(model=ensemble_model,
#                        train_dataloader=train_dataloader,
#                        test_dataloader=test_dataloader,
#                        optimizer=optimizer,
#                        scheduler=scheduler,
#                        loss_fn=loss_fn,
#                        epochs=50,
#                        device=device,
#                        patience=3)

# # End the timer and print out how long it took
# end_time = timer()
# print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")